# Interactive Topology Builder

Build FABRIC topologies programmatically using the new model-based framework.

## Features:
- Type-safe topology construction
- Automatic validation
- Interactive node/network creation
- Export to YAML

## Comparison:
- **Old**: Manual dict manipulation (error-prone)
- **New**: Pydantic models (type-safe, validated)

## Setup

In [ ]:
# Import Setup
import sys
from pathlib import Path

# Add parent directory to Python path to import modules
repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root))

print(f"✅ Python path configured")
print(f"   Repository root: {repo_root}")

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

# Import models
from slice_utils_models import (
    SiteTopology,
    SiteTopologyNodes,
    SiteTopologyNetworks,
    Node,
    NodeCapacity,
    Network,
    SubnetConfig,
    IPv4Config,
    IPv6Config,
    NIC,
    Interface,
    GPU,
    FPGA,
    DPU,
    NVMe,
    PCIDevices,
    NodeSpecific,
    OpenStackRoles,
    PersistentStorage,
    PersistentVolume
)

import yaml

print("✅ Models imported successfully")

In [ ]:
# Define output directory
YAML_OUTPUT_DIR = repo_root / "model"
YAML_OUTPUT_DIR.mkdir(exist_ok=True)
print(f"✅ Output directory: {YAML_OUTPUT_DIR}")

## Initialize Empty Topology

In [ ]:
# Create empty topology
topology = SiteTopology(
    site_topology_nodes=SiteTopologyNodes(nodes={}),
    site_topology_networks=SiteTopologyNetworks(networks={})
)

print("✅ Empty topology created")
print(f"   Nodes: {len(topology.site_topology_nodes.nodes)}")
print(f"   Networks: {len(topology.site_topology_networks.networks)}")

## Add Networks

Create L2 or L3 networks:

In [ ]:
# Add Network 1
net1 = Network(
    name="net_local_1",
    type="L2Bridge",
    subnet="192.168.201.0/24",
    gateway="192.168.201.1"
)

topology.site_topology_networks.networks["net1"] = net1
print(f"✅ Added network: {net1.name}")

In [ ]:
# Add Network 2
net2 = Network(
    name="net_local_2",
    type="L2Bridge",
    subnet="192.168.202.0/24",
    gateway="192.168.202.1"
)

topology.site_topology_networks.networks["net2"] = net2
print(f"✅ Added network: {net2.name}")

## Add Nodes

### Node 1: Full-featured node (GPU, FPGA, NVMe, multiple NICs)

In [ ]:
# Create interfaces for NIC1
nic1_iface1 = Interface(
    device="eth1",
    connection="conn-eth1",
    binding="net_local_1",
    ipv4=IPv4Config(
        address="192.168.201.211/24",
        gateway="192.168.201.1",
        dns="8.8.8.8"
    )
)

# Create NIC1
nic1 = NIC(
    name="nic1",
    model="NIC_Basic",
    interfaces={"iface1": nic1_iface1}
)

# Create interfaces for NIC2
nic2_iface1 = Interface(
    device="eth2",
    connection="conn-eth2",
    binding="net_local_2",
    ipv4=IPv4Config(
        address="192.168.202.211/24",
        gateway="192.168.202.1",
        dns="8.8.8.8"
    )
)

nic2_iface2 = Interface(
    device="eth3",
    connection="conn-eth3",
    binding="",  # Unbound
    ipv4=IPv4Config()
)

# Create NIC2 (ConnectX-6 with 2 interfaces)
nic2 = NIC(
    name="nic2",
    model="NIC_ConnectX_6",
    interfaces={
        "iface1": nic2_iface1,
        "iface2": nic2_iface2
    }
)

# Create GPU
gpu1 = GPU(name="rtx6000", model="GPU_RTX6000")

# Create FPGA
fpga1 = FPGA(name="fpga1", model="FPGA_Xilinx_U280")

# Create NVMe
nvme1 = NVMe(name="nvme1", model="NVME_P4510")

# Create persistent volume
volume1 = PersistentVolume(name="FABRIC_Staff_1T", size=1000)

# Assemble PCI devices
pci_devices = PCIDevices(
    gpu={"gpu1": gpu1},
    fpga={"fpga1": fpga1},
    nvme={"nvme1": nvme1},
    network={"nic1": nic1, "nic2": nic2}
)

# Create persistent storage
storage = PersistentStorage(
    volume={"volume1": volume1}
)

# Create OpenStack roles
openstack_roles = OpenStackRoles(
    control="true",
    network="true",
    compute="false",
    storage="true"
)

# Create node-specific config
node_specific = NodeSpecific(openstack=openstack_roles)

# Create Node 1
node1 = Node(
    name="lc-1",
    hostname="lc-1",
    site="MAX",
    capacity=NodeCapacity(
        cpu=4,
        ram=32,
        disk=100,
        os="default_rocky_9"
    ),
    pci=pci_devices,
    persistent_storage=storage,
    specific=node_specific
)

# Add to topology
topology.site_topology_nodes.nodes["node1"] = node1

print(f"✅ Added node: {node1.hostname}")
print(f"   • CPU: {node1.capacity.cpu} cores")
print(f"   • RAM: {node1.capacity.ram} GB")
print(f"   • NICs: {len(node1.pci.network)}")
print(f"   • GPUs: {len(node1.pci.gpu)}")
print(f"   • FPGAs: {len(node1.pci.fpga)}")
print(f"   • NVMe: {len(node1.pci.nvme)}")

### Node 2: Simple compute node

In [ ]:
# Create simple node with just one NIC
nic1 = NIC(
    name="nic1",
    model="NIC_Basic",
    interfaces={
        "iface1": Interface(
            device="eth1",
            connection="conn-eth1",
            binding="net_local_1",
            ipv4=IPv4Config(
                address="192.168.201.212/24",
                gateway="192.168.201.1",
                dns="8.8.8.8"
            )
        )
    }
)

node2 = Node(
    name="lc-2",
    hostname="lc-2",
    site="MAX",
    capacity=NodeCapacity(
        cpu=2,
        ram=8,
        disk=10,
        os="default_rocky_9"
    ),
    pci=PCIDevices(network={"nic1": nic1}),
    specific=NodeSpecific(
        openstack=OpenStackRoles(
            control="false",
            network="false",
            compute="true",
            storage="false"
        )
    )
)

topology.site_topology_nodes.nodes["node2"] = node2
print(f"✅ Added node: {node2.hostname}")

### Node 3: Another compute node

In [ ]:
# Create node 3 (similar to node 2)
nic1 = NIC(
    name="nic1",
    model="NIC_Basic",
    interfaces={
        "iface1": Interface(
            device="eth1",
            connection="conn-eth1",
            binding="net_local_1",
            ipv4=IPv4Config(
                address="192.168.201.213/24",
                gateway="192.168.201.1",
                dns="8.8.8.8"
            )
        )
    }
)

node3 = Node(
    name="lc-3",
    hostname="lc-3",
    site="MAX",
    capacity=NodeCapacity(
        cpu=2,
        ram=8,
        disk=10,
        os="default_rocky_9"
    ),
    pci=PCIDevices(network={"nic1": nic1}),
    specific=NodeSpecific(
        openstack=OpenStackRoles(
            control="false",
            network="false",
            compute="true",
            storage="false"
        )
    )
)

topology.site_topology_nodes.nodes["node3"] = node3
print(f"✅ Added node: {node3.hostname}")

## Validate Topology

Automatic validation happens when creating models, but we can also explicitly validate:

In [ ]:
try:
    # This will raise ValidationError if invalid
    topology_dict = topology.dict()
    
    print("✅ Topology is valid!")
    print(f"\n📊 Summary:")
    print(f"   • Nodes: {len(topology.site_topology_nodes.nodes)}")
    print(f"   • Networks: {len(topology.site_topology_networks.networks)}")
    
    # Count hardware
    total_gpus = sum(len(n.pci.gpu) for n in topology.site_topology_nodes.nodes.values())
    total_fpgas = sum(len(n.pci.fpga) for n in topology.site_topology_nodes.nodes.values())
    total_nvmes = sum(len(n.pci.nvme) for n in topology.site_topology_nodes.nodes.values())
    
    print(f"   • GPUs: {total_gpus}")
    print(f"   • FPGAs: {total_fpgas}")
    print(f"   • NVMe: {total_nvmes}")
    
except Exception as e:
    print(f"❌ Validation failed: {e}")

## View Topology

Use the viewer to see what we created:

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

import slice_topology_viewer as viewer

# Print compact summary
viewer.print_compact_summary(topology)

In [ ]:
# Print detailed summary
viewer.print_topology_summary(topology)

In [ ]:
# Draw graph
viewer.draw_topology_graph(topology, show_ip=True)

## Export to YAML

In [ ]:
# Export to YAML
output_filename = "_site_topology_generated.yaml"

with open(output_filename, "w") as f:
    yaml.dump(topology.dict(), f, sort_keys=False, default_flow_style=False)

print(f"✅ Topology exported to: {output_filename}")

In [ ]:
# Optional: Add summary header to YAML
viewer.inject_summary_into_yaml_file(output_filename, topology, backup=True)
print(f"✅ Summary added to YAML file")

## Helper Functions

### Quick Node Creation

In [ ]:
def quick_node(
    hostname,
    site="MAX",
    cpu=2,
    ram=8,
    disk=10,
    os="default_rocky_9",
    network_name="net_local_1",
    ipv4_address="192.168.201.100/24",
    is_compute=True
):
    """
    Quickly create a basic node with one NIC.
    """
    nic = NIC(
        name="nic1",
        model="NIC_Basic",
        interfaces={
            "iface1": Interface(
                device="eth1",
                connection="conn-eth1",
                binding=network_name,
                ipv4=IPv4Config(
                    address=ipv4_address,
                    gateway=ipv4_address.rsplit('.', 1)[0] + '.1',
                    dns="8.8.8.8"
                )
            )
        }
    )
    
    node = Node(
        name=hostname,
        hostname=hostname,
        site=site,
        capacity=NodeCapacity(cpu=cpu, ram=ram, disk=disk, os=os),
        pci=PCIDevices(network={"nic1": nic}),
        specific=NodeSpecific(
            openstack=OpenStackRoles(
                compute="true" if is_compute else "false"
            )
        )
    )
    
    return node

# Example: Add a quick node
# node4 = quick_node("lc-4", ipv4_address="192.168.201.214/24")
# topology.site_topology_nodes.nodes["node4"] = node4

### Modify Existing Node

In [ ]:
# Example: Increase RAM on node2
node2_ref = topology.site_topology_nodes.nodes["node2"]
node2_ref.capacity.ram = 16

print(f"✅ Updated node2 RAM to {node2_ref.capacity.ram} GB")

### Remove Node

In [ ]:
# Example: Remove a node
# del topology.site_topology_nodes.nodes["node3"]
# print("✅ Removed node3")

## Summary

This notebook demonstrates the **new type-safe approach** to building topologies:

### Old Approach (Dict-based):
```python
# Error-prone, no validation
node = {
    "name": "lc-1",
    "capacity": {"cpu": 4, "ram": 32},  # Typos not caught
    "pci": {"network": {}}
}
```

### New Approach (Model-based):
```python
# Type-safe, automatic validation
node = Node(
    name="lc-1",
    capacity=NodeCapacity(cpu=4, ram=32),  # Validated!
    pci=PCIDevices()
)
```

### Benefits:
- ✅ **Type Safety** - IDE autocomplete and type checking
- ✅ **Automatic Validation** - Catch errors immediately
- ✅ **Better Error Messages** - Know exactly what's wrong
- ✅ **Cleaner Code** - More readable and maintainable

### Next Steps:
1. Save your topology to YAML
2. Load it in the deployment notebook
3. Deploy to FABRIC!